# Improved lookup of We are Public events

In [3]:
import biu from ..BIU
import json
import IPython
import pandas as pd



SyntaxError: invalid syntax (<ipython-input-3-e55d78269148>, line 1)

In [109]:
class WeArePublic(object):
    
    class Event(object):
        def __init__(self, data):
            self.data = data
            self.extracted = {}
        #edef
        
        def __getattr__(self, key):
            if key in self.data:
                return self.data[key]
            #fi
            raise ValueError
        #edef
        
        @property
        def url(self):
            return self.data['permalink']
        #edef

        @property
        def permdata(self):
            if 'permdata' not in self.extracted:
                d = open(biu.utils.Acquire().curl(self.url).acquire(), 'r').read()
                self.extracted['permdata'] = d
            #fi
            return self.extracted['permdata']
        #edef
        
        @property
        def date(self):
            if 'date' not in self.extracted:
                self.extracted['date'] = self.data['date']['day'].split('>')[-1]
            #fi
            return self.extracted['date']
        
        @property
        def day(self):
            if 'day' not in self.extracted:
                self.extracted['day'] = int(self.date.split(' ')[0])
            #fi
            return self.extracted['day']
        #edef
        
        @property
        def month(self):
            if 'month' not in self.extracted:
                self.extracted['month'] = self.date.split(' ')[1]
            #fi
            return self.extracted['month']
        #edef
        
        @property
        def category(self):
            cat = [ c for c in self.data['lang_term_names'] if c is not None ] if ('lang_term_names' in self.data) else []
            return cat
        #edef
        
        @property
        def location(self):
            return self.data['location']['location']
        #edef
        
        @property
        def city(self):
            return self.data['location']['city']
        #edef
        
        @property
        def price(self):
            if 'price' not in self.extracted:
                lines = [line for line in self.permdata.split('\n') if 'meta' not in line]
                relLine = [ line for line in lines if ('voor We Are Public-leden' in line) ]
                if len(relLine) == 0:
                    relLine = [ line for line in lines if ('toegang' in line) ]
                #fi
                
                if len(relLine) == 0:
                    relLine = [ 'vrij' ]
                #fi
                relLine = relLine[0].lower()
                price = 0
                if 'vrij' not in relLine:
                    price = relLine.split()[0].split('>')[-1].strip()
                    if price in ['vrij', 'vrije', 'exclusief']:
                        price = 0
                    else:
                        price = float(price[1:].replace(',', '.'))
                    #fi
                #fi
                self.extracted['price'] = price
            #fi
            return self.extracted['price']
        #edef                
        
        @property
        def bookable(self):
            if 'bookable' not in self.extracted:
                b = 'Kaartje reserveren' in self.permdata
                self.extracted['bookable'] = b
            #fi
            return self.extracted['bookable']
        #edef
        
        def __getitem__(self, key):
            return self.data[key]
        #edef
        
        #def _repr_pretty_(self, *pargs):
        #    return self.data['thumbnail']['event']
        #edef
        
        def _repr_html_(self):
            print("NOW IN REPR HTML")
            return '<div>' + self.data['thumbnail']['event'] + '</div>'
        #edef
        
        def __repr__(self):
            return str(self)
        #edef
        
        def __str__(self):
            dstr  = "WAP Event\n"
            dstr += " Title:    %s\n" % self.title
            dstr += " Date:     %s\n" % self.date
            dstr += " Location: %s\n" % self.location
            dstr += " City:     %s\n" % self.city
            dstr += " Bookable: %s\n" % ('Yes' if self.bookable else 'No')
            dstr += " URL:      %s\n" % self.url
            dstr += " Category: %s\n" % ' | '.join(self.category)
            return dstr
        #edef
        
    #eclass
    
    def __init__(self, events=None, redo=False):
        if events is None:
            events = self.__retrieveEvents(redo)
        #fi
        self.events = events
        
        self.__idx = { e.id : i for (i,e) in enumerate(self.events) }
    #edef
    
    def __retrieveEvents(self, redo):
        eventList = []
        for page in range(1,15):
            ao = biu.utils.Acquire(redo=redo).curl('https://www.wearepublic.nl/wp-json/wearepublic/v1/events/?page=%d&search=' % page)
            eventList.extend(json.load(open(ao.acquire(),'r')))
        #efor
        eventList = biu.processing.lst.uniq(eventList, key=lambda x: x['permalink'])
        return [ self.Event(data) for data in eventList]
    #edef
    
    def bookables(self):
        return WeArePublic(events=[ e for e in self.events if e.bookable ])
    #edef
    
    def query(self, city=None,
              day=None,
              month=None,
              category=None):
        """Retrieve only events which match certain criteria
        Inputs: city: String or lists of strings of city names
                day: Integer or list of integers (range object possible)
                month: String or lists of strings of months (jan|feb|mar etc)
                category: String or lists of strings of categories"""
        city     = [ city ] if isinstance(city, str) else city
        day      = [ day ] if isinstance(day, int) else day
        month    = [ month ] if isinstance(month, str) else month
        category = [ category ] if isinstance(category, str) else category
        
        relEvents = [ e for e in self.events if
                        ( (city is None)     or (e.city in city)       ) and
                        ( (day is None)      or (e.day in day)         ) and
                        ( (month is None)    or (e.month in month)     ) and
                        ( (category is None) or (e.category in category) ) ]
        
        return WeArePublic(relEvents )
    #edef
    
    @property
    def categories(self):
        return [ c for c in 
                  biu.processing.lst.uniq(biu.processing.lst.flatten([ e.category for e in self.events ]))
                 if c is not None ]
    #edef
    
    @property
    def months(self):
        return biu.processing.lst.uniq([ e.month for e in self.events])
    #edef
    
    @property
    def calendar(self):
        import pandas as pd
        cal = { m : [ "" for i in range(31) ] for m in self.months }
        for e in self.events:
            cal[e.month][e.day-1] = cal[e.month][e.day-1] + ' ' + str(e.id)
        #efor
        df = pd.DataFrame.from_dict(cal)
        df.index = df.index+1
        return df
    #edef
    
    @property
    def table(self):
        data = [ ( e.id,
                   'Yes' if e.bookable else 'No', 
                   '|'.join(e.category),
                   e.city,
                   e.price,
                   e.date,
                   e.title) for e in self.events ]
        return pd.DataFrame(data, columns=['ID', 'Bookable', 'Category', 'City', 'Price', 'Date', 'Title']).set_index('ID')
    #edef
        
    
    def __getitem__(self, key):
        return self.events[self.__idx[key]]
    #edef
    
    def _repr_html_(self):
        return self.table._repr_html_()
    #edef
    
    def __repr__(self):
        return str(self)
    #edef
    
    def __str__(self):
        dstr = "WeArePublic Events\n"
        dstr += '\n'.join([ ' (%5d)%s <%s> %s' % (e.id, '[B]' if e.bookable else '   ', e.date, e.title) for (i,e) in enumerate(self.events) ])
        return dstr
    #edef
    
#eclass
        


wap = WeArePublic()

In [102]:
wap.query(city='Leiden')._repr_html_()

'<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Bookable</th>\n      <th>Category</th>\n      <th>City</th>\n      <th>Price</th>\n      <th>Date</th>\n      <th>Title</th>\n    </tr>\n    <tr>\n      <th>ID</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>74588</th>\n      <td>No</td>\n      <td>Film</td>\n      <td>Leiden</td>\n      <td>0</td>\n      <td>18 aug</td>\n      <td>Trianon klassiekers | Billy Wilder | Ace in th...</td>\n    </tr>\n    <tr>\n      <th>75823</th>\n      <td>Yes</td>\n      <td>Theater</td>\n      <td>Leiden</td>\n      <td>0</td>\n     

In [111]:
wap[75812]._repr_html_()

NOW IN REPR HTML


'<div><img src="https://www.wearepublic.nl/wp-content/uploads/2018/07/pinokkio-720x432.jpg" class="attachment-event_item_image size-event_item_image wp-post-image" alt="" srcset="https://www.wearepublic.nl/wp-content/uploads/2018/07/pinokkio-720x432.jpg 720w, https://www.wearepublic.nl/wp-content/uploads/2018/07/pinokkio-300x180.jpg 300w, https://www.wearepublic.nl/wp-content/uploads/2018/07/pinokkio-768x461.jpg 768w, https://www.wearepublic.nl/wp-content/uploads/2018/07/pinokkio-1024x614.jpg 1024w, https://www.wearepublic.nl/wp-content/uploads/2018/07/pinokkio-500x300.jpg 500w, https://www.wearepublic.nl/wp-content/uploads/2018/07/pinokkio.jpg 1034w" sizes="(max-width: 720px) 100vw, 720px" /></div>'

In [60]:
relLine = [ line for line in wap[75209].permdata.split('\n') if 'voor We Are Public-leden' in line ]

In [61]:
relLine

[]

In [89]:
print(wap[77154])

WAP Event
 Title:    As You Like It
 Date:     29 sep
 Location: Annatheater
 City:     Helmond
 Bookable: No
 URL:      https://www.wearepublic.nl/event/as-you-like-it/
 Category: Theater



In [113]:
type(wap._repr_html_())

str